<a id="back_to_top">

# Ibis Cheat Sheet

#### Table of Contents

- [Obtaining built-in sample data sets](#sample_data_sets)
- [Reading a local csv file](#csv)
- [Viewing schema](#schema)
- [Number of rows and columns](#rows_columns)
- [Selecting certain columns](#selecting_columns)
- [Creating new column based on value from another column using mutate and split() combination](#split)
- [Creating new column using CASE statements](#case)
- [Aggregations / Summarizations](#aggregations)
- [Running Total / Cumulative Sum](#running_total)
- [Renaming a column](#renaming)
- [Forward fill / backward fill](https://ibis-project.org/posts/ffill-and-bfill-using-ibis/index.html)
- [Finding maximum value across all groups](#argmax)
- [Filtering](#filtering)
- [Convert Pandas dataframe to Ibis table expression](#pandas_to_ibis)
- [Convert Ibis table expression to Pandas DataFrame](#ibis_to_pandas)
- [Obtain distinct values in column(s)](#distinct)
- [Find Duplicate Values](#duplicates)
- [Percent of Rows](#perc_of_rows)
- [Percent of Columns](#perc_of_columns)
- [Adding Leading Zeros](#zfill)
- [Creating column values from column names](https://www.youtube.com/watch?v=-0pjPE6VgDs#t=17m)
- [Unnest an array](https://www.youtube.com/watch?v=J7sEn9VklKY#t=7m)
- [Custom Array Functions](https://www.youtube.com/watch?v=6TgpRMmvNQs)
- Additional resources:
    - Youtube Videos: [Phillip in the Cloud](https://www.youtube.com/@cpcloud)

In [1]:
import ibis
import ibis.selectors as s
import pandas as pd
from ibis import _
ibis.options.interactive = True

# create a DuckDB client
client = ibis.duckdb.connect()

<a id="sample_data_sets">

## Obtaining built-in sample data sets

[[back to top]](#back_to_top)

In [2]:
from ibis.interactive import ex

#### To get a list of available data sets

In [3]:
dir(ex)[:20]   # The list is quite long, so limited to just 20

['Aids2',
 'Aids2_raw',
 'Alfalfa',
 'Alfalfa_raw',
 'AllstarFull',
 'AllstarFull_raw',
 'Appearances',
 'Appearances_raw',
 'Assay',
 'Assay_raw',
 'AwardsManagers',
 'AwardsManagers_raw',
 'AwardsPlayers',
 'AwardsPlayers_raw',
 'AwardsShareManagers',
 'AwardsShareManagers_raw',
 'AwardsSharePlayers',
 'AwardsSharePlayers_raw',
 'BOD',
 'BOD_raw']

In [4]:
ex.relig_income_raw

relig_income_raw(name='relig_income_raw', help='Pew religion and income survey')

In [5]:
relig_income = ex.relig_income_raw.fetch()

In [6]:
relig_income

┏━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┓
┃ religion                ┃ <$10k ┃ $10-20k ┃ $20-30k ┃ $30-40k ┃ $40-50k ┃ $50-75k ┃ $75-100k ┃ $100-150k ┃ >150k ┃ Don't know/refused ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━━━━━━━━┩
│ string                  │ int64 │ int64   │ int64   │ int64   │ int64   │ int64   │ int64    │ int64     │ int64 │ int64              │
├─────────────────────────┼───────┼─────────┼─────────┼─────────┼─────────┼─────────┼──────────┼───────────┼───────┼────────────────────┤
│ Agnostic                │    27 │      34 │      60 │      81 │      76 │     137 │      122 │       109 │    84 │                 96 │
│ Atheist                 │    12 │      27 │      37 │      52 │      35 │      70 │       73 │        59 │    74 │                 76 │
│ Buddhist                │    27 │      21 │      30 │      34 │      33 │      58 │       62 │        39 │    53 │                 54 │
│ Catholic                │   418 │     617 │     732 │     670 │     638 │    1116 │      949 │       792 │   633 │               1489 │
│ Don’t know/refused      │    15 │      14 │      15 │      11 │      10 │      35 │       21 │        17 │    18 │                116 │
│ Evangelical Prot        │   575 │     869 │    1064 │     982 │     881 │    1486 │      949 │       723 │   414 │               1529 │
│ Hindu                   │     1 │       9 │       7 │       9 │      11 │      34 │       47 │        48 │    54 │                 37 │
│ Historically Black Prot │   228 │     244 │     236 │     238 │     197 │     223 │      131 │        81 │    78 │                339 │
│ Jehovah's Witness       │    20 │      27 │      24 │      24 │      21 │      30 │       15 │        11 │     6 │                 37 │
│ Jewish                  │    19 │      19 │      25 │      25 │      30 │      95 │       69 │        87 │   151 │                162 │
│ …                       │     … │       … │       … │       … │       … │       … │        … │         … │     … │                  … │
└─────────────────────────┴───────┴─────────┴─────────┴─────────┴─────────┴─────────┴──────────┴───────────┴───────┴────────────────────┘

<a id="csv">

## Reading a CSV file

[[back to top]](#back_to_top)

In [7]:
# read in a local CSV file as an Ibis table
cars = client.read_csv('./data/cars.csv')

In [8]:
cars.head()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━┓
┃ Car                       ┃ MPG     ┃ Cylinders ┃ Displacement ┃ Horsepower ┃ Weight  ┃ Acceleration ┃ Model ┃ Origin ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━┩
│ string                    │ float64 │ int64     │ float64      │ float64    │ float64 │ float64      │ int64 │ string │
├───────────────────────────┼─────────┼───────────┼──────────────┼────────────┼─────────┼──────────────┼───────┼────────┤
│ Chevrolet Chevelle Malibu │    18.0 │         8 │        307.0 │      130.0 │  3504.0 │         12.0 │    70 │ US     │
│ Buick Skylark 320         │    15.0 │         8 │        350.0 │      165.0 │  3693.0 │         11.5 │    70 │ US     │
│ Plymouth Satellite        │    18.0 │         8 │        318.0 │      150.0 │  3436.0 │         11.0 │    70 │ US     │
│ AMC Rebel SST             │    16.0 │         8 │        304.0 │      150.0 │  3433.0 │         12.0 │    70 │ US     │
│ Ford Torino               │    17.0 │         8 │        302.0 │      140.0 │  3449.0 │         10.5 │    70 │ US     │
└───────────────────────────┴─────────┴───────────┴──────────────┴────────────┴─────────┴──────────────┴───────┴────────┘

<a id="schema">

## Viewing Schema

[[back to top]](#back_to_top)

In [9]:
cars.schema()

ibis.Schema {
  Car           string
  MPG           float64
  Cylinders     int64
  Displacement  float64
  Horsepower    float64
  Weight        float64
  Acceleration  float64
  Model         int64
  Origin        string
}

<a id="rows_columns">

## Number of Rows and Columns

[[back to top]](#back_to_top)

Number of rows:

In [10]:
cars.count()

406

Number of Columns:

In [11]:
len(cars.schema())

9

<a id="selecting_columns">

## Selecting Certain Columns

[[back to top]](#back_to_top)

#### We can use pandas square bracket syntax

In [12]:
cars['Origin','Car','MPG'].head()

┏━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┓
┃ Origin ┃ Car                       ┃ MPG     ┃
┡━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━┩
│ string │ string                    │ float64 │
├────────┼───────────────────────────┼─────────┤
│ US     │ Chevrolet Chevelle Malibu │    18.0 │
│ US     │ Buick Skylark 320         │    15.0 │
│ US     │ Plymouth Satellite        │    18.0 │
│ US     │ AMC Rebel SST             │    16.0 │
│ US     │ Ford Torino               │    17.0 │
└────────┴───────────────────────────┴─────────┘

#### or use ibis `select()` method

In [13]:
cars.select('Origin', 'Car', 'MPG').head()

┏━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┓
┃ Origin ┃ Car                       ┃ MPG     ┃
┡━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━┩
│ string │ string                    │ float64 │
├────────┼───────────────────────────┼─────────┤
│ US     │ Chevrolet Chevelle Malibu │    18.0 │
│ US     │ Buick Skylark 320         │    15.0 │
│ US     │ Plymouth Satellite        │    18.0 │
│ US     │ AMC Rebel SST             │    16.0 │
│ US     │ Ford Torino               │    17.0 │
└────────┴───────────────────────────┴─────────┘

#### Select columns based on their data types

In [14]:
from ibis import selectors as s
cars.select(s.of_type('float64'))

┏━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ MPG     ┃ Displacement ┃ Horsepower ┃ Weight  ┃ Acceleration ┃
┡━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━┩
│ float64 │ float64      │ float64    │ float64 │ float64      │
├─────────┼──────────────┼────────────┼─────────┼──────────────┤
│    18.0 │        307.0 │      130.0 │  3504.0 │         12.0 │
│    15.0 │        350.0 │      165.0 │  3693.0 │         11.5 │
│    18.0 │        318.0 │      150.0 │  3436.0 │         11.0 │
│    16.0 │        304.0 │      150.0 │  3433.0 │         12.0 │
│    17.0 │        302.0 │      140.0 │  3449.0 │         10.5 │
│    15.0 │        429.0 │      198.0 │  4341.0 │         10.0 │
│    14.0 │        454.0 │      220.0 │  4354.0 │          9.0 │
│    14.0 │        440.0 │      215.0 │  4312.0 │          8.5 │
│    14.0 │        455.0 │      225.0 │  4425.0 │         10.0 │
│    15.0 │        390.0 │      190.0 │  3850.0 │          8.5 │
│       … │            … │          … │       … │            … │
└─────────┴──────────────┴────────────┴─────────┴──────────────┘

#### We can also use boolean logic OR using `|` to select columns with different column types

In [15]:
cars.select(s.of_type('string') | s.of_type('int64'))

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━┓
┃ Car                       ┃ Cylinders ┃ Model ┃ Origin ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━┩
│ string                    │ int64     │ int64 │ string │
├───────────────────────────┼───────────┼───────┼────────┤
│ Chevrolet Chevelle Malibu │         8 │    70 │ US     │
│ Buick Skylark 320         │         8 │    70 │ US     │
│ Plymouth Satellite        │         8 │    70 │ US     │
│ AMC Rebel SST             │         8 │    70 │ US     │
│ Ford Torino               │         8 │    70 │ US     │
│ Ford Galaxie 500          │         8 │    70 │ US     │
│ Chevrolet Impala          │         8 │    70 │ US     │
│ Plymouth Fury iii         │         8 │    70 │ US     │
│ Pontiac Catalina          │         8 │    70 │ US     │
│ AMC Ambassador DPL        │         8 │    70 │ US     │
│ …                         │         … │     … │ …      │
└───────────────────────────┴───────────┴───────┴────────┘

#### We can also select based on range of column indices using special `s.r` object

In [16]:
cars.select(s.r[:3])

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┓
┃ Car                       ┃ MPG     ┃ Cylinders ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━┩
│ string                    │ float64 │ int64     │
├───────────────────────────┼─────────┼───────────┤
│ Chevrolet Chevelle Malibu │    18.0 │         8 │
│ Buick Skylark 320         │    15.0 │         8 │
│ Plymouth Satellite        │    18.0 │         8 │
│ AMC Rebel SST             │    16.0 │         8 │
│ Ford Torino               │    17.0 │         8 │
│ Ford Galaxie 500          │    15.0 │         8 │
│ Chevrolet Impala          │    14.0 │         8 │
│ Plymouth Fury iii         │    14.0 │         8 │
│ Pontiac Catalina          │    14.0 │         8 │
│ AMC Ambassador DPL        │    15.0 │         8 │
│ …                         │       … │         … │
└───────────────────────────┴─────────┴───────────┘

<a id="split">

## Creating new column based on value from another column using mutate and split() combination

[[back to top]](#back_to_top)

The `Car` column happens to contain both the car's make and model names.  If we want the make, it looks like we just need to extract the first string token after the splitting the string value in the `Car` column based on a single white space.  The model name is the 2nd token.  We will use array index of zero/0 to obtain the make and index of 1 to obtain the model name.

In [17]:
cars = cars.mutate(Make=cars.Car.split(' ')[0])
cars = cars.mutate(ModelName=cars.Car.split(' ')[1])

In [18]:
cars.head()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┓
┃ Car                       ┃ MPG     ┃ Cylinders ┃ Displacement ┃ Horsepower ┃ Weight  ┃ Acceleration ┃ Model ┃ Origin ┃ Make      ┃ ModelName ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━┩
│ string                    │ float64 │ int64     │ float64      │ float64    │ float64 │ float64      │ int64 │ string │ string    │ string    │
├───────────────────────────┼─────────┼───────────┼──────────────┼────────────┼─────────┼──────────────┼───────┼────────┼───────────┼───────────┤
│ Chevrolet Chevelle Malibu │    18.0 │         8 │        307.0 │      130.0 │  3504.0 │         12.0 │    70 │ US     │ Chevrolet │ Chevelle  │
│ Buick Skylark 320         │    15.0 │         8 │        350.0 │      165.0 │  3693.0 │         11.5 │    70 │ US     │ Buick     │ Skylark   │
│ Plymouth Satellite        │    18.0 │         8 │        318.0 │      150.0 │  3436.0 │         11.0 │    70 │ US     │ Plymouth  │ Satellite │
│ AMC Rebel SST             │    16.0 │         8 │        304.0 │      150.0 │  3433.0 │         12.0 │    70 │ US     │ AMC       │ Rebel     │
│ Ford Torino               │    17.0 │         8 │        302.0 │      140.0 │  3449.0 │         10.5 │    70 │ US     │ Ford      │ Torino    │
└───────────────────────────┴─────────┴───────────┴──────────────┴────────────┴─────────┴──────────────┴───────┴────────┴───────────┴───────────┘

or we can use "method chaining" syntax to use a single mutate function

In [19]:
cars.mutate(
    Make=cars.Car.split(' ')[0],
    ModelName=cars.Car.split(' ')[1]
)

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Car                       ┃ MPG     ┃ Cylinders ┃ Displacement ┃ Horsepower ┃ Weight  ┃ Acceleration ┃ Model ┃ Origin ┃ Make      ┃ ModelName  ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ string                    │ float64 │ int64     │ float64      │ float64    │ float64 │ float64      │ int64 │ string │ string    │ string     │
├───────────────────────────┼─────────┼───────────┼──────────────┼────────────┼─────────┼──────────────┼───────┼────────┼───────────┼────────────┤
│ Chevrolet Chevelle Malibu │    18.0 │         8 │        307.0 │      130.0 │  3504.0 │         12.0 │    70 │ US     │ Chevrolet │ Chevelle   │
│ Buick Skylark 320         │    15.0 │         8 │        350.0 │      165.0 │  3693.0 │         11.5 │    70 │ US     │ Buick     │ Skylark    │
│ Plymouth Satellite        │    18.0 │         8 │        318.0 │      150.0 │  3436.0 │         11.0 │    70 │ US     │ Plymouth  │ Satellite  │
│ AMC Rebel SST             │    16.0 │         8 │        304.0 │      150.0 │  3433.0 │         12.0 │    70 │ US     │ AMC       │ Rebel      │
│ Ford Torino               │    17.0 │         8 │        302.0 │      140.0 │  3449.0 │         10.5 │    70 │ US     │ Ford      │ Torino     │
│ Ford Galaxie 500          │    15.0 │         8 │        429.0 │      198.0 │  4341.0 │         10.0 │    70 │ US     │ Ford      │ Galaxie    │
│ Chevrolet Impala          │    14.0 │         8 │        454.0 │      220.0 │  4354.0 │          9.0 │    70 │ US     │ Chevrolet │ Impala     │
│ Plymouth Fury iii         │    14.0 │         8 │        440.0 │      215.0 │  4312.0 │          8.5 │    70 │ US     │ Plymouth  │ Fury       │
│ Pontiac Catalina          │    14.0 │         8 │        455.0 │      225.0 │  4425.0 │         10.0 │    70 │ US     │ Pontiac   │ Catalina   │
│ AMC Ambassador DPL        │    15.0 │         8 │        390.0 │      190.0 │  3850.0 │          8.5 │    70 │ US     │ AMC       │ Ambassador │
│ …                         │       … │         … │            … │          … │       … │            … │     … │ …      │ …         │ …          │
└───────────────────────────┴─────────┴───────────┴──────────────┴────────────┴─────────┴──────────────┴───────┴────────┴───────────┴────────────┘

<a id="case">

## Creating new column using CASE statements

[[back to top]](#back_to_top)

In [20]:
foods = client.read_csv('./data/food.csv')

In [21]:
foods

┏━━━━━━━━━━━━━┳━━━━━━━━━┓
┃ food        ┃ ounces  ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━┩
│ string      │ float64 │
├─────────────┼─────────┤
│ bacon       │     4.0 │
│ pulled pork │     3.0 │
│ bacon       │    12.0 │
│ pastrami    │     6.0 │
│ corned beef │     7.5 │
│ bacon       │     8.0 │
│ pastrami    │     3.0 │
│ honey ham   │     5.0 │
│ nova lox    │     6.0 │
└─────────────┴─────────┘

In [22]:
case = (
    foods.food
    .case()
    .when('bacon', 'pig')
    .when('pulled pork', 'pig')
    .when('pastrami', 'cow')
    .when('corned beef', 'cow')
    .when('honey ham', 'pig')
    .else_('salmon')
    .end()
)

foods = foods.mutate(animal=case)

In [23]:
foods

┏━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┓
┃ food        ┃ ounces  ┃ animal ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━┩
│ string      │ float64 │ string │
├─────────────┼─────────┼────────┤
│ bacon       │     4.0 │ pig    │
│ pulled pork │     3.0 │ pig    │
│ bacon       │    12.0 │ pig    │
│ pastrami    │     6.0 │ cow    │
│ corned beef │     7.5 │ cow    │
│ bacon       │     8.0 │ pig    │
│ pastrami    │     3.0 │ cow    │
│ honey ham   │     5.0 │ pig    │
│ nova lox    │     6.0 │ salmon │
└─────────────┴─────────┴────────┘

<a id="aggregations">

## Aggregations / Summarizations

[[back to top]](#back_to_top)

In [24]:
cars.MPG.max()

46.6

In [25]:
cars['MPG'].max()

46.6

It may be surprising that you can't perform min, max, etc after doing a `select()`

In [26]:
cars.select('MPG').max()

AttributeError: 'Table' object has no attribute 'max'

**GOAL:** By country of origin, calculate min mpg, max mpg, and avg mpg.

**NOTE:** If you want to group by more than one column, need to pass in a list of column names

In [27]:
agged = cars.group_by('Origin').aggregate(
    min_mpg=cars['MPG'].min(),
    max_mpg=cars['MPG'].max(),
    avg_mpg=cars['MPG'].mean(),
)
agged

┏━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┓
┃ Origin ┃ min_mpg ┃ max_mpg ┃ avg_mpg   ┃
┡━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━┩
│ string │ float64 │ float64 │ float64   │
├────────┼─────────┼─────────┼───────────┤
│ Japan  │    18.0 │    46.6 │ 30.450633 │
│ US     │     0.0 │    39.0 │ 19.688189 │
│ Europe │     0.0 │    44.3 │ 26.745205 │
└────────┴─────────┴─────────┴───────────┘

<a id="renaming">

## Renaming column names

[[back to top]](#back_to_top)

In case you want to rename the columns:

In [28]:
agged.rename(
    min_miles_per_gallon='min_mpg',
    max_miles_per_gallon='max_mpg',
    avg_miles_per_gallon='avg_mpg',
)

┏━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ Origin ┃ min_miles_per_gallon ┃ max_miles_per_gallon ┃ avg_miles_per_gallon ┃
┡━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│ string │ float64              │ float64              │ float64              │
├────────┼──────────────────────┼──────────────────────┼──────────────────────┤
│ US     │                  0.0 │                 39.0 │            19.688189 │
│ Europe │                  0.0 │                 44.3 │            26.745205 │
│ Japan  │                 18.0 │                 46.6 │            30.450633 │
└────────┴──────────────────────┴──────────────────────┴──────────────────────┘

Using a window function like `row_number()` to obtain the top 3 most fuel efficient car from each Origin/Country:

In [29]:
(
    cars
    .mutate(rank=ibis.row_number().over(group_by=_.Origin, order_by=ibis.desc(_.MPG)) + 1)
    .order_by(['Origin', ibis.desc('MPG')])
    .filter(_.rank <= 3)
)

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━┓
┃ Car                               ┃ MPG     ┃ Cylinders ┃ Displacement ┃ Horsepower ┃ Weight  ┃ Acceleration ┃ Model ┃ Origin ┃ Make       ┃ ModelName ┃ rank  ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━┩
│ string                            │ float64 │ int64     │ float64      │ float64    │ float64 │ float64      │ int64 │ string │ string     │ string    │ int64 │
├───────────────────────────────────┼─────────┼───────────┼──────────────┼────────────┼─────────┼──────────────┼───────┼────────┼────────────┼───────────┼───────┤
│ Volkswagen Rabbit C (Diesel)      │    44.3 │         4 │         90.0 │       48.0 │  2085.0 │         21.7 │    80 │ Europe │ Volkswagen │ Rabbit    │     1 │
│ Volkswagen Pickup                 │    44.0 │         4 │         97.0 │       52.0 │  2130.0 │         24.6 │    82 │ Europe │ Volkswagen │ Pickup    │     2 │
│ Volkswagen Dasher (diesel)        │    43.4 │         4 │         90.0 │       48.0 │  2335.0 │         23.7 │    80 │ Europe │ Volkswagen │ Dasher    │     3 │
│ Mazda GLC                         │    46.6 │         4 │         86.0 │       65.0 │  2110.0 │         17.9 │    80 │ Japan  │ Mazda      │ GLC       │     1 │
│ Honda Civic 1500 gl               │    44.6 │         4 │         91.0 │       67.0 │  1850.0 │         13.8 │    80 │ Japan  │ Honda      │ Civic     │     2 │
│ Datsun 210                        │    40.8 │         4 │         85.0 │       65.0 │  2110.0 │         19.2 │    80 │ Japan  │ Datsun     │ 210       │     3 │
│ Plymouth Champ                    │    39.0 │         4 │         86.0 │       64.0 │  1875.0 │         16.4 │    81 │ US     │ Plymouth   │ Champ     │     1 │
│ Plymouth Horizon Miser            │    38.0 │         4 │        105.0 │       63.0 │  2125.0 │         14.7 │    82 │ US     │ Plymouth   │ Horizon   │     2 │
│ Oldsmobile Cutlass Ciera (diesel) │    38.0 │         6 │        262.0 │       85.0 │  3015.0 │         17.0 │    82 │ US     │ Oldsmobile │ Cutlass   │     3 │
└───────────────────────────────────┴─────────┴───────────┴──────────────┴────────────┴─────────┴──────────────┴───────┴────────┴────────────┴───────────┴───────┘

<a id="running_total">

## Running Total / Cumulative Sum

[[back to top]](#back_to_top)

Let's calculate running total or cumulative sum of MPG by country of origin.

If you have a background in SQL and familiar with window functions, below is defining our "window" to order and group our data by:

In [30]:
# Define running total calculation
window_spec = ibis.window(
    order_by='MPG',
    group_by='Origin'
)

Then we will calcuate our running total of MPG (cumulative sum) utilizing the window we just defined above, the cumulative sum will be in a column called "running_total"

In [31]:
running_total = cars['MPG'].cumsum().over(window_spec).name('running_total')

In [32]:
running_total

┏━━━━━━━━━━━━━━━┓
┃ running_total ┃
┡━━━━━━━━━━━━━━━┩
│ float64       │
├───────────────┤
│        2405.6 │
│        2405.6 │
│        2405.6 │
│        2405.6 │
│        2405.6 │
│        2405.6 │
│        2405.6 │
│        2405.6 │
│        2405.6 │
│        2405.6 │
│             … │
└───────────────┘

Apply running_total window spec in the mutate() function.

In [33]:
cars_cumulative_mpg = cars['Origin','Car','MPG'].mutate(running_total=running_total)

Since we can't view the entire results, we will look at results for each Origin and find out which of their respective model had the highest MPG.

In [34]:
cars_cumulative_mpg.filter(cars_cumulative_mpg['Origin'] == 'US')

┏━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Origin ┃ Car                              ┃ MPG     ┃ running_total ┃
┡━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ string │ string                           │ float64 │ float64       │
├────────┼──────────────────────────────────┼─────────┼───────────────┤
│ US     │ Chevrolet Chevelle Concours (sw) │     0.0 │        5000.8 │
│ US     │ Ford Torino (sw)                 │     0.0 │        5000.8 │
│ US     │ Plymouth Satellite (sw)          │     0.0 │        5000.8 │
│ US     │ AMC Rebel SST (sw)               │     0.0 │        5000.8 │
│ US     │ Ford Mustang Boss 302            │     0.0 │        5000.8 │
│ US     │ Hi 1200D                         │     9.0 │        5000.8 │
│ US     │ Ford F250                        │    10.0 │        5000.8 │
│ US     │ Chevy C20                        │    10.0 │        5000.8 │
│ US     │ Dodge D200                       │    11.0 │        5000.8 │
│ US     │ Mercury Marquis                  │    11.0 │        5000.8 │
│ …      │ …                                │       … │             … │
└────────┴──────────────────────────────────┴─────────┴───────────────┘

In [35]:
cars_cumulative_mpg.filter(cars_cumulative_mpg['Origin'] == 'US').filter(_.MPG == _.MPG.max())

┏━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Origin ┃ Car            ┃ MPG     ┃ running_total ┃
┡━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ string │ string         │ float64 │ float64       │
├────────┼────────────────┼─────────┼───────────────┤
│ US     │ Plymouth Champ │    39.0 │        5000.8 │
└────────┴────────────────┴─────────┴───────────────┘

In [36]:
cars_cumulative_mpg.filter(cars_cumulative_mpg['Origin'] == 'Japan')

┏━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Origin ┃ Car                         ┃ MPG     ┃ running_total ┃
┡━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ string │ string                      │ float64 │ float64       │
├────────┼─────────────────────────────┼─────────┼───────────────┤
│ Japan  │ Mazda RX3                   │    18.0 │        2405.6 │
│ Japan  │ Mazda RX2 Coupe             │    19.0 │        2405.6 │
│ Japan  │ Toyota Mark II              │    19.0 │        2405.6 │
│ Japan  │ Toyota Camry                │    20.0 │        2405.6 │
│ Japan  │ Toyota Mark II              │    20.0 │        2405.6 │
│ Japan  │ Toyota Celica GT Liftback   │    21.1 │        2405.6 │
│ Japan  │ Mazda RX-4                  │    21.5 │        2405.6 │
│ Japan  │ Datsun 610                  │    22.0 │        2405.6 │
│ Japan  │ Datsun 810                  │    22.0 │        2405.6 │
│ Japan  │ Toyota Corolla Mark II (sw) │    23.0 │        2405.6 │
│ …      │ …                           │       … │             … │
└────────┴─────────────────────────────┴─────────┴───────────────┘

In [37]:
cars_cumulative_mpg.filter(cars_cumulative_mpg['Origin'] == 'Japan').filter(_.MPG == _.MPG.max())

┏━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Origin ┃ Car       ┃ MPG     ┃ running_total ┃
┡━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ string │ string    │ float64 │ float64       │
├────────┼───────────┼─────────┼───────────────┤
│ Japan  │ Mazda GLC │    46.6 │        2405.6 │
└────────┴───────────┴─────────┴───────────────┘

In [38]:
cars_cumulative_mpg.filter(cars_cumulative_mpg['Origin'] == 'Europe')

┏━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Origin ┃ Car                         ┃ MPG     ┃ running_total ┃
┡━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ string │ string                      │ float64 │ float64       │
├────────┼─────────────────────────────┼─────────┼───────────────┤
│ Europe │ Citroen DS-21 Pallas        │     0.0 │        1952.4 │
│ Europe │ Volkswagen Super Beetle 117 │     0.0 │        1952.4 │
│ Europe │ Saab 900s                   │     0.0 │        1952.4 │
│ Europe │ Peugeot 604sl               │    16.2 │        1952.4 │
│ Europe │ Mercedes-Benz 280s          │    16.5 │        1952.4 │
│ Europe │ Volvo 264gl                 │    17.0 │        1952.4 │
│ Europe │ Volvo 145e (sw)             │    18.0 │        1952.4 │
│ Europe │ Volvo 144ea                 │    19.0 │        1952.4 │
│ Europe │ Peugeot 504                 │    19.0 │        1952.4 │
│ Europe │ Audi 100LS                  │    20.0 │        1952.4 │
│ …      │ …                           │       … │             … │
└────────┴─────────────────────────────┴─────────┴───────────────┘

In [39]:
cars_cumulative_mpg.filter(cars_cumulative_mpg['Origin'] == 'Europe').filter(_.MPG == _.MPG.max())

┏━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Origin ┃ Car                          ┃ MPG     ┃ running_total ┃
┡━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ string │ string                       │ float64 │ float64       │
├────────┼──────────────────────────────┼─────────┼───────────────┤
│ Europe │ Volkswagen Rabbit C (Diesel) │    44.3 │        1952.4 │
└────────┴──────────────────────────────┴─────────┴───────────────┘

<a id="argmax">

## Finding maximum value across all groups

[[back to top]](#back_to_top)

Above, we had to issue 3 filters to find the country with the maximum MPG.  But, what if there's a way to do this with fewer steps?  `argmax()` returns the largest value in a column that also happens to be in groupings.  `argmax()` will then return the largest value regardless of the group.

The country or origin with largest MPG for all origins:

In [40]:
cars_cumulative_mpg.Origin.argmax(cars_cumulative_mpg.MPG)

'Japan'

The criteria above can then be used in our subsequent filtering below to obtain max MPG:

In [41]:
cars_cumulative_mpg.filter(
    (_.Origin == cars_cumulative_mpg.Origin.argmax(cars_cumulative_mpg.MPG))
    & (_.MPG == _.MPG.max())
)

┏━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Origin ┃ Car       ┃ MPG     ┃ running_total ┃
┡━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ string │ string    │ float64 │ float64       │
├────────┼───────────┼─────────┼───────────────┤
│ Japan  │ Mazda GLC │    46.6 │        2405.6 │
└────────┴───────────┴─────────┴───────────────┘

<a id="filtering">

## Filtering

[[back to top]](#back_to_top)

In [42]:
cars

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Car                       ┃ MPG     ┃ Cylinders ┃ Displacement ┃ Horsepower ┃ Weight  ┃ Acceleration ┃ Model ┃ Origin ┃ Make      ┃ ModelName  ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ string                    │ float64 │ int64     │ float64      │ float64    │ float64 │ float64      │ int64 │ string │ string    │ string     │
├───────────────────────────┼─────────┼───────────┼──────────────┼────────────┼─────────┼──────────────┼───────┼────────┼───────────┼────────────┤
│ Chevrolet Chevelle Malibu │    18.0 │         8 │        307.0 │      130.0 │  3504.0 │         12.0 │    70 │ US     │ Chevrolet │ Chevelle   │
│ Buick Skylark 320         │    15.0 │         8 │        350.0 │      165.0 │  3693.0 │         11.5 │    70 │ US     │ Buick     │ Skylark    │
│ Plymouth Satellite        │    18.0 │         8 │        318.0 │      150.0 │  3436.0 │         11.0 │    70 │ US     │ Plymouth  │ Satellite  │
│ AMC Rebel SST             │    16.0 │         8 │        304.0 │      150.0 │  3433.0 │         12.0 │    70 │ US     │ AMC       │ Rebel      │
│ Ford Torino               │    17.0 │         8 │        302.0 │      140.0 │  3449.0 │         10.5 │    70 │ US     │ Ford      │ Torino     │
│ Ford Galaxie 500          │    15.0 │         8 │        429.0 │      198.0 │  4341.0 │         10.0 │    70 │ US     │ Ford      │ Galaxie    │
│ Chevrolet Impala          │    14.0 │         8 │        454.0 │      220.0 │  4354.0 │          9.0 │    70 │ US     │ Chevrolet │ Impala     │
│ Plymouth Fury iii         │    14.0 │         8 │        440.0 │      215.0 │  4312.0 │          8.5 │    70 │ US     │ Plymouth  │ Fury       │
│ Pontiac Catalina          │    14.0 │         8 │        455.0 │      225.0 │  4425.0 │         10.0 │    70 │ US     │ Pontiac   │ Catalina   │
│ AMC Ambassador DPL        │    15.0 │         8 │        390.0 │      190.0 │  3850.0 │          8.5 │    70 │ US     │ AMC       │ Ambassador │
│ …                         │       … │         … │            … │          … │       … │            … │     … │ …      │ …         │ …          │
└───────────────────────────┴─────────┴───────────┴──────────────┴────────────┴─────────┴──────────────┴───────┴────────┴───────────┴────────────┘

When using `filter()` and multiple conditions, you need to wrap each condition with parenthesis.  When using boolean logic, use `&` and `|` symbols for `AND` or `OR` logic, respectively.

In [43]:
cars.filter(
    (cars['MPG'] > 0)
    & (cars['Cylinders'] < 5)
)['MPG'].mean()

29.118750000000002

In [44]:
cars.filter(
    (cars['MPG'] > 0) & (cars['Cylinders'] < 5)
)

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━┓
┃ Car                          ┃ MPG     ┃ Cylinders ┃ Displacement ┃ Horsepower ┃ Weight  ┃ Acceleration ┃ Model ┃ Origin ┃ Make       ┃ ModelName ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━┩
│ string                       │ float64 │ int64     │ float64      │ float64    │ float64 │ float64      │ int64 │ string │ string     │ string    │
├──────────────────────────────┼─────────┼───────────┼──────────────┼────────────┼─────────┼──────────────┼───────┼────────┼────────────┼───────────┤
│ Toyota Corolla Mark ii       │    24.0 │         4 │        113.0 │       95.0 │  2372.0 │         15.0 │    70 │ Japan  │ Toyota     │ Corolla   │
│ Datsun PL510                 │    27.0 │         4 │         97.0 │       88.0 │  2130.0 │         14.5 │    70 │ Japan  │ Datsun     │ PL510     │
│ Volkswagen 1131 Deluxe Sedan │    26.0 │         4 │         97.0 │       46.0 │  1835.0 │         20.5 │    70 │ Europe │ Volkswagen │ 1131      │
│ Peugeot 504                  │    25.0 │         4 │        110.0 │       87.0 │  2672.0 │         17.5 │    70 │ Europe │ Peugeot    │ 504       │
│ Audi 100 LS                  │    24.0 │         4 │        107.0 │       90.0 │  2430.0 │         14.5 │    70 │ Europe │ Audi       │ 100       │
│ Saab 99e                     │    25.0 │         4 │        104.0 │       95.0 │  2375.0 │         17.5 │    70 │ Europe │ Saab       │ 99e       │
│ BMW 2002                     │    26.0 │         4 │        121.0 │      113.0 │  2234.0 │         12.5 │    70 │ Europe │ BMW        │ 2002      │
│ Datsun PL510                 │    27.0 │         4 │         97.0 │       88.0 │  2130.0 │         14.5 │    71 │ Japan  │ Datsun     │ PL510     │
│ Chevrolet Vega 2300          │    28.0 │         4 │        140.0 │       90.0 │  2264.0 │         15.5 │    71 │ US     │ Chevrolet  │ Vega      │
│ Toyota Corolla               │    25.0 │         4 │        113.0 │       95.0 │  2228.0 │         14.0 │    71 │ Japan  │ Toyota     │ Corolla   │
│ …                            │       … │         … │            … │          … │       … │            … │     … │ …      │ …          │ …         │
└──────────────────────────────┴─────────┴───────────┴──────────────┴────────────┴─────────┴──────────────┴───────┴────────┴────────────┴───────────┘

In [45]:
min_mpg = 0
max_cyl = 5
condition = (cars['MPG'] > min_mpg) & (cars['Cylinders'] < max_cyl)

cars.filter(condition)

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━┓
┃ Car                          ┃ MPG     ┃ Cylinders ┃ Displacement ┃ Horsepower ┃ Weight  ┃ Acceleration ┃ Model ┃ Origin ┃ Make       ┃ ModelName ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━┩
│ string                       │ float64 │ int64     │ float64      │ float64    │ float64 │ float64      │ int64 │ string │ string     │ string    │
├──────────────────────────────┼─────────┼───────────┼──────────────┼────────────┼─────────┼──────────────┼───────┼────────┼────────────┼───────────┤
│ Toyota Corolla Mark ii       │    24.0 │         4 │        113.0 │       95.0 │  2372.0 │         15.0 │    70 │ Japan  │ Toyota     │ Corolla   │
│ Datsun PL510                 │    27.0 │         4 │         97.0 │       88.0 │  2130.0 │         14.5 │    70 │ Japan  │ Datsun     │ PL510     │
│ Volkswagen 1131 Deluxe Sedan │    26.0 │         4 │         97.0 │       46.0 │  1835.0 │         20.5 │    70 │ Europe │ Volkswagen │ 1131      │
│ Peugeot 504                  │    25.0 │         4 │        110.0 │       87.0 │  2672.0 │         17.5 │    70 │ Europe │ Peugeot    │ 504       │
│ Audi 100 LS                  │    24.0 │         4 │        107.0 │       90.0 │  2430.0 │         14.5 │    70 │ Europe │ Audi       │ 100       │
│ Saab 99e                     │    25.0 │         4 │        104.0 │       95.0 │  2375.0 │         17.5 │    70 │ Europe │ Saab       │ 99e       │
│ BMW 2002                     │    26.0 │         4 │        121.0 │      113.0 │  2234.0 │         12.5 │    70 │ Europe │ BMW        │ 2002      │
│ Datsun PL510                 │    27.0 │         4 │         97.0 │       88.0 │  2130.0 │         14.5 │    71 │ Japan  │ Datsun     │ PL510     │
│ Chevrolet Vega 2300          │    28.0 │         4 │        140.0 │       90.0 │  2264.0 │         15.5 │    71 │ US     │ Chevrolet  │ Vega      │
│ Toyota Corolla               │    25.0 │         4 │        113.0 │       95.0 │  2228.0 │         14.0 │    71 │ Japan  │ Toyota     │ Corolla   │
│ …                            │       … │         … │            … │          … │       … │            … │     … │ …      │ …          │ …         │
└──────────────────────────────┴─────────┴───────────┴──────────────┴────────────┴─────────┴──────────────┴───────┴────────┴────────────┴───────────┘

<a id="pandas_to_ibis">

## Convert Pandas dataframe to Ibis table expression

[[back to top]](#back_to_top)

In [46]:
data = pd.DataFrame({'k1': ['one'] * 3 + ['two'] * 4 + ['three'] * 1, 'k2': [3, 2, 1, 3, 3, 4, 4, 1]})

In [47]:
data

,k1,k2
0,one,3
1,one,2
2,one,1
3,two,3
4,two,3
5,two,4
6,two,4
7,three,1


In [48]:
ibis_data = ibis.memtable(data)

In [49]:
ibis_data

┏━━━━━━━━┳━━━━━━━┓
┃ k1     ┃ k2    ┃
┡━━━━━━━━╇━━━━━━━┩
│ string │ int64 │
├────────┼───────┤
│ one    │     3 │
│ one    │     2 │
│ one    │     1 │
│ two    │     3 │
│ two    │     3 │
│ two    │     4 │
│ two    │     4 │
│ three  │     1 │
└────────┴───────┘

<a id="ibis_to_pandas">

## Convert Ibis table expression to Pandas DataFrame

[[back to top]](#back_to_top)

In [50]:
pdf = ibis_data.to_pandas()

In [51]:
pdf

,k1,k2
0,one,3
1,one,2
2,one,1
3,two,3
4,two,3
5,two,4
6,two,4
7,three,1


<a id="distinct">

## Obtain distinct values in column(s)

[[back to top]](#back_to_top)

Distinct values in specific column

In [52]:
ibis_data.select("k1").distinct()

┏━━━━━━━━┓
┃ k1     ┃
┡━━━━━━━━┩
│ string │
├────────┤
│ two    │
│ three  │
│ one    │
└────────┘

or across all columns

In [53]:
ibis_data.distinct()

┏━━━━━━━━┳━━━━━━━┓
┃ k1     ┃ k2    ┃
┡━━━━━━━━╇━━━━━━━┩
│ string │ int64 │
├────────┼───────┤
│ one    │     2 │
│ two    │     4 │
│ three  │     1 │
│ one    │     1 │
│ two    │     3 │
│ one    │     3 │
└────────┴───────┘

<a id="duplicates">

## Find Duplicate Values

[[back to top]](#back_to_top)

In [54]:
(
    ibis_data
    .group_by('k1')
    .aggregate(
        Count = _.k1.count()
    )
    .filter(_.Count > 1)
)

┏━━━━━━━━┳━━━━━━━┓
┃ k1     ┃ Count ┃
┡━━━━━━━━╇━━━━━━━┩
│ string │ int64 │
├────────┼───────┤
│ two    │     4 │
│ one    │     3 │
└────────┴───────┘

<a id="perc_of_rows">

## Percent of Rows

[[back to top]](#back_to_top)

In [55]:
data = pd.DataFrame(
    {'group': [80, 70, 75, 75],
     'ounces': [20, 30, 25, 25],
     'size': [100, 100, 100, 100]
    }
)

In [56]:
ibis_data = ibis.memtable(data)

In [57]:
ibis_data

┏━━━━━━━┳━━━━━━━━┳━━━━━━━┓
┃ group ┃ ounces ┃ size  ┃
┡━━━━━━━╇━━━━━━━━╇━━━━━━━┩
│ int64 │ int64  │ int64 │
├───────┼────────┼───────┤
│    80 │     20 │   100 │
│    70 │     30 │   100 │
│    75 │     25 │   100 │
│    75 │     25 │   100 │
└───────┴────────┴───────┘

In [58]:
(
    ibis_data
    .mutate(group_perc=_.group / (_.group + _.ounces + _.size) * 100)
    .mutate(ounces_perc=_.ounces / (_.group + _.ounces + _.size) * 100)
    .mutate(size_perc=_.size / (_.group + _.ounces + _.size) * 100)
    .select(s.contains('perc'))
)

┏━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━┓
┃ group_perc ┃ ounces_perc ┃ size_perc ┃
┡━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━┩
│ float64    │ float64     │ float64   │
├────────────┼─────────────┼───────────┤
│       40.0 │        10.0 │      50.0 │
│       35.0 │        15.0 │      50.0 │
│       37.5 │        12.5 │      50.0 │
│       37.5 │        12.5 │      50.0 │
└────────────┴─────────────┴───────────┘

<a id="perc_of_columns">

## Percent of Columns

[[back to top]](#back_to_top)

In [59]:
(
    ibis_data
    .mutate(group_perc=_.group / (_.group.sum()) * 100)
    .mutate(ounces_perc=_.ounces / (_.ounces.sum()) * 100)
    .mutate(size_perc=_.size / (_.size.sum()) * 100)
    .select(s.contains('perc'))
)

┏━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━┓
┃ group_perc ┃ ounces_perc ┃ size_perc ┃
┡━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━┩
│ float64    │ float64     │ float64   │
├────────────┼─────────────┼───────────┤
│  26.666667 │        20.0 │      25.0 │
│  23.333333 │        30.0 │      25.0 │
│  25.000000 │        25.0 │      25.0 │
│  25.000000 │        25.0 │      25.0 │
└────────────┴─────────────┴───────────┘

<a id="zfill">

## Adding Leading Zeros

[[back to top]](#back_to_top)

pandas has a handy [zfill()](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.zfill.html) method which ibis does not have.  But it is easy relatively easy to implement this using ibis.

In [60]:
us_fips_codes = client.read_csv('data/co-est2020-alldata.csv')

In [61]:
us_fips_codes

┏━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ STATE ┃ COUNTY ┃ STNAME  ┃ CTYNAME         ┃
┡━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ int64 │ int64  │ string  │ string          │
├───────┼────────┼─────────┼─────────────────┤
│     1 │      0 │ Alabama │ Alabama         │
│     1 │      1 │ Alabama │ Autauga County  │
│     1 │      3 │ Alabama │ Baldwin County  │
│     1 │      5 │ Alabama │ Barbour County  │
│     1 │      7 │ Alabama │ Bibb County     │
│     1 │      9 │ Alabama │ Blount County   │
│     1 │     11 │ Alabama │ Bullock County  │
│     1 │     13 │ Alabama │ Butler County   │
│     1 │     15 │ Alabama │ Calhoun County  │
│     1 │     17 │ Alabama │ Chambers County │
│     … │      … │ …       │ …               │
└───────┴────────┴─────────┴─────────────────┘

Goal:
- New column: The state FIPS code is a 2-character code that should have a leading zero when the state code is a single digit code
- New column: The county FIPS codes is a 3-character code that should have 2 leading zeros when the county code is a single digit code, 1 leading zero when it is a 2 digit code

In [62]:
(
    us_fips_codes
    .mutate(state_fips=ibis.literal("0") * (2 - us_fips_codes.STATE.cast('string').length()) + us_fips_codes.STATE.cast('string'))
    .mutate(county_fips=ibis.literal("0") * (3 - us_fips_codes.COUNTY.cast('string').length()) + us_fips_codes.COUNTY.cast('string'))
)

┏━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ STATE ┃ COUNTY ┃ STNAME  ┃ CTYNAME         ┃ state_fips ┃ county_fips ┃
┡━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ int64 │ int64  │ string  │ string          │ string     │ string      │
├───────┼────────┼─────────┼─────────────────┼────────────┼─────────────┤
│     1 │      0 │ Alabama │ Alabama         │ 01         │ 000         │
│     1 │      1 │ Alabama │ Autauga County  │ 01         │ 001         │
│     1 │      3 │ Alabama │ Baldwin County  │ 01         │ 003         │
│     1 │      5 │ Alabama │ Barbour County  │ 01         │ 005         │
│     1 │      7 │ Alabama │ Bibb County     │ 01         │ 007         │
│     1 │      9 │ Alabama │ Blount County   │ 01         │ 009         │
│     1 │     11 │ Alabama │ Bullock County  │ 01         │ 011         │
│     1 │     13 │ Alabama │ Butler County   │ 01         │ 013         │
│     1 │     15 │ Alabama │ Calhoun County  │ 01         │ 015         │
│     1 │     17 │ Alabama │ Chambers County │ 01         │ 017         │
│     … │      … │ …       │ …               │ …          │ …           │
└───────┴────────┴─────────┴─────────────────┴────────────┴─────────────┘